In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=bae732bd3b2313254cd7af3fb97cdbf2a13a27e8c14e158bfd3a7ffbbb02124d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql.functions import trim, to_date, year, month
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
#Leitura da base
df_exame = spark.read.csv('./sample_data/exame.csv', header= True)

In [ ]:
#Leitura da base
df_atestado = spark.read.csv('./sample_data/atestado.csv', header= True)

In [ ]:
#Leitura da base
df_empresa = spark.read.csv('./sample_data/empresa.csv', header= True)

In [ ]:
#Leitura da base
df_funcionario = spark.read.csv('./sample_data/funcionario.csv', header= True)

In [ ]:
#Leitura da base
df_medico = spark.read.csv('./sample_data/medico.csv', header= True)

In [ ]:
#Leitura da base
df_tipoExame = spark.read.csv('./sample_data/tipoExame.csv', header= True)

**Quantidade de pacientes de uma empresa que são atendidos pelo médico X**
- CNPJ da empresa: 80699840108863
- Médico escolhido (crm): 961013

In [ ]:
teste_medico = df_medico.filter(F.col('crm') == "961013")

In [ ]:
teste_empresa = df_funcionario.filter(F.col('cnpj_empresa') == '80699840108863')

In [ ]:
teste_exame = df_exame.filter(F.col('crm_medico') == "961013")

In [ ]:
teste_funcionario = df_empresa.filter(F.col("cnpj") == '80699840108863')

In [ ]:
cond = [teste_empresa.cnpj_empresa == teste_funcionario.cnpj]

In [ ]:
paciente_join_empresa = df_funcionario.join(df_empresa, cond, "inner")

In [ ]:
paciente_join_empresa.show()

+-----------+---------------+-------+-------------------+--------------+--------------+---------+--------------------+--------+
|        cpf|           nome|  cargo|      data_admissao|  cnpj_empresa|          cnpj|     nome|            endereco| contato|
+-----------+---------------+-------+-------------------+--------------+--------------+---------+--------------------+--------+
|22280567316| Funcionario 98|Cargo 9|2016-07-01 00:00:00|80699840108863|80699840108863|Empresa 1|Endereco a1421358...|87136308|
|77321735610| Funcionario 99|Cargo 8|2015-09-03 00:00:00|80699840108863|80699840108863|Empresa 1|Endereco a1421358...|87136308|
|43597340271|Funcionario 100|Cargo 3|2021-07-16 00:00:00|80699840108863|80699840108863|Empresa 1|Endereco a1421358...|87136308|
|17534355968|Funcionario 101|Cargo 6|2021-10-02 00:00:00|80699840108863|80699840108863|Empresa 1|Endereco a1421358...|87136308|
|38394232387|Funcionario 102|Cargo 5|2015-09-26 00:00:00|80699840108863|80699840108863|Empresa 1|Enderec

In [ ]:
teste_medico = df_medico.filter(F.col('crm') == '961013')

In [ ]:
teste_exame = df_exame.filter(F.col('crm_medico') == '961013')

In [ ]:
cond2 = [teste_medico.crm == teste_exame.crm_medico]

In [ ]:
medico_join_exame = df_exame.join(df_medico, cond2, "inner")

In [ ]:
medico_join_exame.show()

+-----+--------------------+-------------+----------+---------------+------+--------+--------+--------------------+
|   id|           resultado|id_tipo_exame|crm_medico|cpf_funcionario|   crm|    nome| contato|            endereco|
+-----+--------------------+-------------+----------+---------------+------+--------+--------+--------------------+
|82161|a5015d487dd85d27a...|            1|    499854|    49351573837|499854|Medico 9|77390599|28efc602bd5ce8b81...|
|82162|30bab50df9ee2bf24...|            1|    499854|    81363226505|499854|Medico 9|77390599|28efc602bd5ce8b81...|
|82163|8c8d130ee70c21bfd...|            1|    499854|    68254741145|499854|Medico 9|77390599|28efc602bd5ce8b81...|
|82164|e36745d6bdeecbbf1...|            1|    499854|    64175550881|499854|Medico 9|77390599|28efc602bd5ce8b81...|
|82165|18e1c36bc8adf382f...|            1|    499854|    43847696529|499854|Medico 9|77390599|28efc602bd5ce8b81...|
|82166|6027bd570af64e390...|            1|    499854|    45860585765|499

In [ ]:
cond3 = [paciente_join_empresa.cpf == medico_join_exame.cpf_funcionario]

In [ ]:
qtd_pac_medico = medico_join_exame.join(paciente_join_empresa, cond3, "inner")

In [ ]:
qtd_pac_medico.count()

1000000

In [ ]:
qtd_pac_medico.show()

+-----+--------------------+-------------+----------+---------------+------+--------+--------+--------------------+-----------+---------------+--------+-------------------+--------------+--------------+---------+--------------------+--------+
|   id|           resultado|id_tipo_exame|crm_medico|cpf_funcionario|   crm|    nome| contato|            endereco|        cpf|           nome|   cargo|      data_admissao|  cnpj_empresa|          cnpj|     nome|            endereco| contato|
+-----+--------------------+-------------+----------+---------------+------+--------+--------+--------------------+-----------+---------------+--------+-------------------+--------------+--------------+---------+--------------------+--------+
|82502|b29b7ac6be207c403...|            1|    499854|    99006160212|499854|Medico 9|77390599|28efc602bd5ce8b81...|99006160212|Funcionario 599| Cargo 9|2020-09-01 00:00:00|34654281088150|34654281088150|Empresa 3|Endereco 6dd139a7...|33846789|
|82543|27143fd3eff3717aa...|

**Tipo de exame mais feito por determinada empresa**
- Empresa (cnpj): 80699840108863

In [ ]:
cond4 = [paciente_join_empresa.cpf == df_exame.cpf_funcionario]

In [ ]:
tipo_exame = paciente_join_empresa.join(df_exame, cond, "inner")

In [ ]:
tipo_exame.show()

+-----------+--------------+-------+-------------------+--------------+--------------+---------+--------------------+--------+-----+--------------------+-------------+----------+---------------+
|        cpf|          nome|  cargo|      data_admissao|  cnpj_empresa|          cnpj|     nome|            endereco| contato|   id|           resultado|id_tipo_exame|crm_medico|cpf_funcionario|
+-----------+--------------+-------+-------------------+--------------+--------------+---------+--------------------+--------+-----+--------------------+-------------+----------+---------------+
|22280567316|Funcionario 98|Cargo 9|2016-07-01 00:00:00|80699840108863|80699840108863|Empresa 1|Endereco a1421358...|87136308|82161|a5015d487dd85d27a...|            1|    499854|    49351573837|
|22280567316|Funcionario 98|Cargo 9|2016-07-01 00:00:00|80699840108863|80699840108863|Empresa 1|Endereco a1421358...|87136308|82162|30bab50df9ee2bf24...|            1|    499854|    81363226505|
|22280567316|Funcionario 

In [ ]:
tipo_exame.groupby('id_tipo_exame').count().show()